We import the corresponding libraries

In [1]:
import os
import shutil
from collections import OrderedDict

import json
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
import torch

We check the GPU specifications


In [11]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-24066ff8-89c9-8615-6237-41ecdf4e1939)


We check the CPU specifications

In [12]:
!lscpu |grep 'Model name'

Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz


We install the nn-UNet V2 library and restar the runtime after instalation

In [ ]:
!pip install nnunetv2

We import the nn-UNet V2 libary

In [2]:
import nnunetv2

We conect to the google drive containing all the data files

In [3]:
# for colab users only - mounting the drive

from google.colab import drive
drive.mount('/content/drive',force_remount = True)

drive_dir = "/content/drive/My Drive"
mount_dir = os.path.join(drive_dir, "Colab Notebooks")
base_dir = os.getcwd()

Mounted at /content/drive


In [4]:
assert os.path.exists(drive_dir)
if os.path.exists(mount_dir) is False:
    os.makedirs(mount_dir)

We create an set the different evironment paths necessary for the nnU-Net to work

In [5]:
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input: 
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder 
    """
    if os.path.exists(folder_path):
        
        if not overwrite:
            print(f"{folder_path} exists.")
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created.")

In [ ]:
# Maybe move path of preprocessed data directly on content - this may be signifcantely faster!
print("Current Working Directory {}".format(os.getcwd()))
path_dict = {
    "nnUNet_raw" : os.path.join(mount_dir, "nnUNet_raw"), 
    "nnUNet_preprocessed" : os.path.join(mount_dir, "nnUNet_preprocessed"), # 1 experiment: 1 epoch took 112s
    "nnUNet_results" : os.path.join(mount_dir, "nnUNet_results"),
}

# Write paths to environment variables
for env_var, path in path_dict.items():
  os.environ[env_var] = path 

# Check whether all environment variables are set correct!
for env_var, path in path_dict.items():
  if os.getenv(env_var) != path:
    print("Error:")
    print("Environment Variable {} is not set correctly!".format(env_var))
    print("Should be {}".format(path))
    print("Variable is {}".format(os.getenv(env_var)))
  make_if_dont_exist(path, overwrite=False)

print("No Error Occured, Continue Forward.")

In [9]:
!set nnUNet_raw="/content/drive/My Drive/Colab Notebooks/nnUNet_raw"
!set nnUNet_preprocessed="/content/drive/My Drive/Colab Notebooks/nnUNet_preprocessed"
!set nnUNet_results="/content/drive/My Drive/Colab Notebooks/nnUNet_results"

We lounch the data preprocessing

In [ ]:
!nnUNetv2_plan_and_preprocess -d 111 

We train the nnUnet in 3D full resolution mode for the 5 different folders

In [ ]:
!nnUNetv2_train 111 3d_fullres 0 --npz

In [ ]:
!nnUNetv2_train 111 3d_fullres 1 --npz

In [ ]:
!nnUNetv2_train 111 3d_fullres 2 --npz

In [ ]:
!nnUNetv2_train 111 3d_fullres 3 --npz

In [ ]:
!nnUNetv2_train 111 3d_fullres 4 --npz

We find the best configuration

In [ ]:
!nnUNetv2_find_best_configuration 111 -c 3d_fullres 

We run inference using the model generated from the folder 1, which yielded the best results

In [ ]:
!nnUNetv2_predict -i "/content/drive/My Drive/Colab Notebooks/test" -o "/content/drive/My Drive/Colab Notebooks/test_pred" -d 111 -c 3d_fullres -f 1
